In [15]:
import pandas as pd

df = pd.read_json('output-with-chat-gpt-label.json')

saveDf = pd.DataFrame(columns=['context', 'question', 'llama-subproblems', 'label', 'chatgpt-guess-label'])

#for index, row in df.iterrows():
#    new_record = {
#            'question': row['question'],
#            'subproblems': row['subproblems'],
#            'label': row['label'],
#        }
#    temp_df = pd.DataFrame([new_record])
#    saveDf = pd.concat([saveDf, temp_df], ignore_index=True)
#saveDf.to_json('./output-subproblems-parsed.json', orient='records')

In [16]:
import re
def fetch_subquestions_array(text):
    pattern = r'.*\?'
    lines = text.split('\n')
    matching_lines = []
    for line in lines:
        line = line.strip()
        if re.match(pattern, line):
            matching_lines.append(line)

    
    return matching_lines


text = df.iloc[9]['subproblems']
text
#print(text)
print(fetch_subquestions_array(text))



['1. Is the crab less than three years old?', '2. Does the crab destroy the wall constructed by the frog?', '3. Does the frog enjoy the company of the crow?']


In [11]:
i = 0
avgQuestions = 0
for index, row in df.iterrows():
    v = fetch_subquestions_array(row['subproblems'])
    v = list(set(v))
    avgQuestions += len(v)
    if len(v) == 0:
        i += 1
print(i, avgQuestions / 1000)

167 3.737


In [17]:
based_substring = " Based on the game state and the rules and preferences,"

df.iloc[0]['question'].split(based_substring)[-1].strip()

'does the swan swear to the woodpecker?'

In [18]:
for index, row in df.iterrows():
    v = fetch_subquestions_array(row['subproblems'])
    s = set()
    k = []
    for i in v:
        if i not in s:
            k.append(i)
        s.add(i)
    
    based_substring = " Based on the game state and the rules and preferences,"

    new_record = {
            'context': row['question'].split(based_substring)[0],
            'question': f'{based_substring.strip()} {row["question"].split(based_substring)[-1].strip()}',
            'llama-subproblems': '||'.join(k),
            'label': row['label'],
            'chatgpt-guess-label': row['chatgpt-guess-label']
        }
    temp_df = pd.DataFrame([new_record])
    saveDf = pd.concat([saveDf, temp_df], ignore_index=True)
saveDf.to_json('./output-subproblems-parsed-2.json', orient='records')